<a href="https://colab.research.google.com/github/zacharywendholt/CSCI-B455-Final/blob/master/pokemonBattle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import sklearn
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, Flatten
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

pokemonURL = 'https://raw.githubusercontent.com/zacharywendholt/CSCI-B455-Final/master/pokemon-challenge/pokemon.csv'
pokemonData = pd.read_csv(pokemonURL)


In [0]:
#re-assigning names to these columns because they have spaces in their name
pokemonData.rename(columns={"Type 1": "Type1"}, inplace=True)
pokemonData.rename(columns={"Type 2": "Type2"}, inplace=True)
# order of legendary pokemon is not randomized. Need to make sure we randomize 
# when we test we need to make sure there is a mix of both legendary and not legendary
cols = pokemonData.columns[2:len(pokemonData.columns)-1]

#changing the string values for type to integer values
le = preprocessing.LabelEncoder()
pokemonData.Type1 = le.fit_transform(pokemonData.Type1)

#type 2 has null values so fill them with na
pokemonData.Type2 = pokemonData.Type2.fillna(method= "ffill")
pokemonData.Type2 = le.fit_transform(pokemonData.Type2)

x = pokemonData.loc[:, cols]
y = pokemonData.Legendary

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =0.3, random_state=42)



In [0]:
model = Sequential()
model.add(Dense(15, input_dim = len(cols), activation='softmax'))
model.add(Dense(1, input_dim = 15, activation = 'relu'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics =['accuracy'])

In [175]:
model.fit(x_train,y_train, epochs=10, batch_size = 32, validation_data= (x_test, y_test))

_, accuracy = model.evaluate(x_test,y_test)

print("Accuracy: ",accuracy)


Train on 560 samples, validate on 240 samples
Epoch 1/10
560/560 [==============================] - 0s 220us/step - loss: 0.7603 - accuracy: 0.6625 - val_loss: 0.5526 - val_accuracy: 0.7000
Epoch 2/10
560/560 [==============================] - 0s 57us/step - loss: 0.9434 - accuracy: 0.7214 - val_loss: 0.5311 - val_accuracy: 0.7125
Epoch 3/10
560/560 [==============================] - 0s 53us/step - loss: 0.8774 - accuracy: 0.7286 - val_loss: 0.4712 - val_accuracy: 0.7167
Epoch 4/10
560/560 [==============================] - 0s 51us/step - loss: 0.7789 - accuracy: 0.8179 - val_loss: 0.4599 - val_accuracy: 0.9375
Epoch 5/10
560/560 [==============================] - 0s 57us/step - loss: 0.6428 - accuracy: 0.9107 - val_loss: 0.4697 - val_accuracy: 0.9375
Epoch 6/10
560/560 [==============================] - 0s 50us/step - loss: 0.6191 - accuracy: 0.9107 - val_loss: 0.4483 - val_accuracy: 0.9375
Epoch 7/10
560/560 [==============================] - 0s 50us/step - loss: 0.6010 - accuracy: 0